In [1]:
!pip install langchain langchain_community openai PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 16.2 MB/s eta 0:00:00


In [2]:
import os
import json
import pandas as pd
from langchain_community.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [18]:
#from dotenv import load_dotenv
#load_dotenv
#key = os.getenv("OPENAI_API_KEY")
key = "sk-proj-yfwtusS1nUs0upQbH6MaT3BlbkFJlZnHCF3vHa8xzszwLgXI"
llm = ChatOpenAI(openai_api_key = key, model_name = 'gpt-3.5-turbo', temperature = 0.5)

## **Quiz generation**

In [19]:
response_json = {
    "1": {
        "mcq" : "multiple choice question",
        "options" : {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here"
        },
        "correct" : "correct answer"
    },
    "2" : {
        "mcq" : "multiple choice question",
        "options" : {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here"
        },
        "correct" : "correct answer"
    },
    "3" : {
        "mcq" : "multiple choice question",
        "options" : {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here"
        },
        "correct": "correct answer"
    }
}

In [20]:
template = """
Text : {text}
You are an expert MCQ maker. Given the above text, it is your job to
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like response_json below  and use it as a guide.
Ensure to make {number} MCQs.
# response_json
{response_json}
"""

In [21]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ['text', 'number', 'subject', 'tone', 'response_json'],
    template = template
)

In [22]:
#We create a LLMChain to connect the prompt and llm
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = 'quiz', verbose = True)

# **Quiz evaluation**

The MCQs generated by the quiz_chain are evaluated using the evaluate chain

In [23]:
template2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.
if the quiz is not at per with the cognitive and analytical abilities of the students,
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}
Check from an expert English Writer of the above quiz:
"""

In [24]:
quiz_evaluation_prompt = PromptTemplate(input_variables = ["subject", "quiz"], template = template2)

In [25]:
review_chain = LLMChain(llm = llm, prompt = quiz_evaluation_prompt, output_key = "review", verbose = True)

We chain the quiz_chain and review_chain using a SequentialChain that takes in the required inputs and outputs the quiz and its review

In [26]:
generate_evaluate_chain = SequentialChain(chains = [quiz_chain, review_chain], input_variables = ["text", "number", "subject", "tone", "response_json"],
                                        output_variables = ["quiz", "review"], verbose = True)

## **Chain inputs**

### **Text input**

In [27]:
file_path = "/content/machine_learning.txt"

In [28]:
with open(file_path, 'r') as file:
    text = file.read()

In [29]:
print(text)

Machine learning approaches are traditionally divided into three broad categories, which correspond to learning paradigms, depending on the nature of the "signal" or "feedback" available to the learning system:

Supervised learning: The computer is presented with example inputs and their desired outputs, given by a "teacher", and the goal is to learn a general rule that maps inputs to outputs.
Unsupervised learning: No labels are given to the learning algorithm, leaving it on its own to find structure in its input. Unsupervised learning can be a goal in itself (discovering hidden patterns in data) or a means towards an end (feature learning).
Reinforcement learning: A computer program interacts with a dynamic environment in which it must perform a certain goal (such as driving a vehicle or playing a game against an opponent). As it navigates its problem space, the program is provided feedback that's analogous to rewards, which it tries to maximize.
Although each algorithm has advantage

### **Response JSON Input**

In [30]:
#Serializing the python dictionary into a JSON-formatted string
json.dumps(response_json)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [31]:
number = 10
subject = "Machine Learning"
tone = "simple"

## **Inference**

In [32]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#using get_openai_callback we can do Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text" : text,
            "number" : number,
            "subject" : subject,
            "tone" : tone,
            "response_json" : json.dumps(response_json)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text : Machine learning approaches are traditionally divided into three broad categories, which correspond to learning paradigms, depending on the nature of the "signal" or "feedback" available to the learning system:

Supervised learning: The computer is presented with example inputs and their desired outputs, given by a "teacher", and the goal is to learn a general rule that maps inputs to outputs.
Unsupervised learning: No labels are given to the learning algorithm, leaving it on its own to find structure in its input. Unsupervised learning can be a goal in itself (discovering hidden patterns in data) or a means towards an end (feature learning).
Reinforcement learning: A computer program interacts with a dynamic environment in which it must perform a certain goal (such as driving a vehicle or playing a game against an opponent). As it navigates its problem space, the program is pr

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## **Token Usage Tracking**

In [35]:
print(f"Total Tokens:{cb.total_tokens}") #prompt_tokens + completion_tokens i.e. input tokens + output tokens
print(f"Prompt Tokens:{cb.prompt_tokens}") #Input token
print(f"Completion Tokens:{cb.completion_tokens}") #Output token
print(f"Total Cost:{cb.total_cost}") #In dollars

Total Tokens:0
Prompt Tokens:0
Completion Tokens:0
Total Cost:0.0


## **Output after formatting inference response**

In [ ]:
response
'''
{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Donald Hebb, a Canadian psychologist, published a book titled The Organization of Behavior where he introduces the Hebbian theory, discussing the neural structure or synapses between the nerve cells.[14] Hebb’s model of neurons interacting with one another sets a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied on human cognitive systems contributed to the modern machine learning technologies as well including logician Walter Pitts and Warren McCulloch, who proposed the early mathematical models of neural networks to come up with algorithms that mirror human thought processes.[13]\n\nBy the early 1960s an experimental "learning machine" with punched tape memory, called Cybertron, had been developed by Raytheon Company to analyze sonar signals, electrocardiograms, and speech patterns using rudimentary reinforcement learning. It was repetitively "trained" by a human operator/teacher to recognize patterns and equipped with a "goof" button to cause it to re-evaluate incorrect decisions.[15] A representative book on research into machine learning during the 1960s was Nilsson\'s book on Learning Machines, dealing mostly with machine learning for pattern classification.[16] Interest related to pattern recognition continued into the 1970s, as described by Duda and Hart in 1973.[17] In 1981 a report was given on using teaching strategies so that a neural network learns to recognize 40 characters (26 letters, 10 digits, and 4 special symbols) from a computer terminal.[18]\n\nTom M. Mitchell provided a widely quoted, more formal definition of the algorithms studied in the machine learning field: "A computer program is said to learn from experience E with respect to some class of tasks T and performance measure P if its performance at tasks in T, as measured by P, improves with experience E."[19] This definition of the tasks in which machine learning is concerned offers a fundamentally operational definition rather than defining the field in cognitive terms. This follows Alan Turing\'s proposal in his paper "Computing Machinery and Intelligence", in which the question "Can machines think?" is replaced with the question "Can machines do what we (as thinking entities) can do?".[20]\n\nModern-day machine learning has two objectives, one is to classify data based on models which have been developed, the other purpose is to make predictions for future outcomes based on these models. A hypothetical algorithm specific to classifying data may use computer vision of moles coupled with supervised learning in order to train it to classify the cancerous moles. A machine learning algorithm for stock trading may inform the trader of future potential predictions.[21]',
 'number': 5,
 'subject': 'machine learning',
 'tone': 'simple',
 'response_json': '{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}',
 'quiz': '{"1": {"mcq": "Who coined the term machine learning?", "options": {"a": "Donald Hebb", "b": "Arthur Samuel", "c": "Walter Pitts", "d": "Warren McCulloch"}, "correct": "b"}, "2": {"mcq": "What was the earliest machine learning model introduced by Arthur Samuel?", "options": {"a": "Speech recognition", "b": "Image classification", "c": "Checkers game", "d": "Pattern recognition"}, "correct": "c"}, "3": {"mcq": "Which book introduced the Hebbian theory?", "options": {"a": "The Organization of Behavior", "b": "Learning Machines", "c": "Computing Machinery and Intelligence", "d": "The History of Machine Learning"}, "correct": "a"}, "4": {"mcq": "In the 1960s, a learning machine called Cybertron was developed to analyze which of the following?", "options": {"a": "Sonar signals", "b": "Speech patterns", "c": "Electrocardiograms", "d": "All of the above"}, "correct": "d"}, "5": {"mcq": "According to Tom M. Mitchell, what is the definition of machine learning?", "options": {"a": "Improving computer performance", "b": "Learning from experience to improve task performance", "c": "Analyzing cognitive processes", "d": "Developing neural networks"}, "correct": "b"}}',
 'review': '{"1": {"mcq": "Who coined the term \'machine learning\'?", "options": {"a": "Donald Hebb", "b": "Arthur Samuel", "c": "Walter Pitts", "d": "Warren McCulloch"}, "correct": "b"}, "2": {"mcq": "What was the earliest machine learning model introduced by Arthur Samuel?", "options": {"a": "Speech recognition", "b": "Image classification", "c": "Checkers game", "d": "Pattern recognition"}, "correct": "c"}, "3": {"mcq": "Which book introduced the Hebbian theory?", "options": {"a": "The Organization of Behavior", "b": "Learning Machines", "c": "Computing Machinery and Intelligence", "d": "Cybertron"}, "correct": "a"}, "4": {"mcq": "What is the purpose of modern-day machine learning?", "options": {"a": "To classify data based on developed models", "b": "To predict future outcomes based on models", "c": "Both a and b", "d": "Neither a nor b"}, "correct": "c"}, "5": {"mcq": "What is the operational definition of machine learning according to Tom M. Mitchell?", "options": {"a": "Improving performance with experience", "b": "Classifying data based on models", "c": "Making predictions for future outcomes", "d": "Performing tasks that humans can do"}, "correct": "a"}}'}
 '''

In [ ]:
quiz = response.get("quiz")
quiz = json.loads(quiz)

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option} : {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ" : mcq, "Choices" : options, "Correct" : correct})

''' quiz_table_data :
[{'MCQ': 'Who coined the term machine learning?',
  'Choices': 'a: Donald Hebb | b: Arthur Samuel | c: Walter Pitts | d: Warren McCulloch',
  'Correct': 'b'},
 {'MCQ': 'What was the earliest machine learning model introduced by Arthur Samuel?',
  'Choices': 'a: Speech recognition | b: Image classification | c: Checkers game | d: Pattern recognition',
  'Correct': 'c'},
 {'MCQ': 'Which book introduced the Hebbian theory?',
  'Choices': 'a: The Organization of Behavior | b: Learning Machines | c: Computing Machinery and Intelligence | d: The History of Machine Learning',
  'Correct': 'a'},
 {'MCQ': 'In the 1960s, a learning machine called Cybertron was developed to analyze which of the following?',
  'Choices': 'a: Sonar signals | b: Speech patterns | c: Electrocardiograms | d: All of the above',
  'Correct': 'd'},
 {'MCQ': 'According to Tom M. Mitchell, what is the definition of machine learning?',
  'Choices': 'a: Improving computer performance | b: Learning from experience to improve task performance | c: Analyzing cognitive processes | d: Developing neural networks',
  'Correct': 'b'}]
'''


## **Saving the quiz**

In [ ]:
quiz = pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning.csv", index = False)